In [21]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import time


In [22]:
df = pd.read_csv("Trips_2018.csv")

#Remove unwanted index column if it exists
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

#Convert start and stop times to proper datetime format
df["starttime"] = pd.to_datetime(df["starttime"], errors="coerce")
df["stoptime"]  = pd.to_datetime(df.get("stoptime"), errors="coerce")


#Create a new column 'start_hour' — the trip’s start time rounded down to the hour (used later to join with hourly weather data)
df["start_hour"] = df["starttime"].dt.floor("H")

#Compute trip duration in minutes
df["trip_duration_min"] = (
    df["tripduration"]/60.0
    if "tripduration" in df.columns
    else (df["stoptime"] - df["starttime"]).dt.total_seconds()/60
)

#Find a representative latitude and longitude for weather data Using median coordinates ensures one central location (e.g., NYC center)
LAT, LON = df["start_station_latitude"].median(), df["start_station_longitude"].median()

#Find the overall date range of the dataset for API query
START = df["start_hour"].min().date().isoformat()
END   = df["start_hour"].max().date().isoformat()

#Print a quick summary
print(f"Trips: {len(df)}  |  Date span: {START} → {END}")

C:\Users\UPASANA\AppData\Local\Temp\ipykernel_52460\1714329481.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["start_hour"] = df["starttime"].dt.floor("H")


Trips: 17548339  |  Date span: 2018-01-01 → 2018-12-31


In [23]:
# --- Replace everything from here (old per-station loop) ---

# 1) Small spatial grid to reduce API calls (≈2–3 km cells)
GRID = 0.03  # increase to 0.05 for fewer calls, decrease to 0.02 for more detail
stations = stations.copy()
stations["lat_bin"] = (stations["start_station_latitude"]  / GRID).round().astype(int)
stations["lon_bin"] = (stations["start_station_longitude"] / GRID).round().astype(int)
stations["cell_id"] = stations["lat_bin"].astype(str) + "_" + stations["lon_bin"].astype(str)

cells = (stations.groupby("cell_id", as_index=False)
         .agg(lat=("start_station_latitude", "median"),
              lon=("start_station_longitude","median")))

print(f"Unique grid cells to fetch: {len(cells)}")

def fetch_cell(lat, lon, start_date, end_date, retries=3, pause=1.0):
    url = "https://archive-api.open-meteo.com/v1/era5"
    params = {
        "latitude": float(lat),
        "longitude": float(lon),
        "start_date": start_date,
        "end_date": end_date,
        "timezone": "America/New_York",
        "hourly": [
            "temperature_2m","apparent_temperature","rain","snowfall",
            "wind_speed_10m","relative_humidity_2m","cloud_cover","visibility"
        ]
    }
    for a in range(retries):
        try:
            resp = requests.get(url, params=params, timeout=30)
            js = resp.json()
            if resp.ok and "hourly" in js:
                wx = pd.DataFrame(js["hourly"])
                wx["start_hour"] = pd.to_datetime(wx["time"])
                wx = wx.drop(columns=["time"])
                # memory friendly
                for c in wx.columns:
                    if c != "start_hour" and pd.api.types.is_float_dtype(wx[c]):
                        wx[c] = pd.to_numeric(wx[c], downcast="float")
                return wx
            time.sleep(pause * (a+1))  # backoff then retry
        except Exception:
            time.sleep(pause * (a+1))
    return None

# 2) Fetch once per cell
cell_weather = []
for _, r in cells.iterrows():
    wx_cell = fetch_cell(r["lat"], r["lon"], START, END)
    if wx_cell is None:
        print(f"⚠️  Cell {r['cell_id']} failed")
        continue
    wx_cell["cell_id"] = r["cell_id"]
    cell_weather.append(wx_cell)
    time.sleep(0.2)  

if not cell_weather:
    raise RuntimeError("No weather could be fetched. Try increasing GRID or check network.")

wx_cells = pd.concat(cell_weather, ignore_index=True)
print("Weather rows (cells):", len(wx_cells), "| cells covered:", wx_cells["cell_id"].nunique())

# 3) Expand cell weather back to each station in that cell
wx = (stations[["start_station_id","cell_id"]]
      .merge(wx_cells, on="cell_id", how="left"))

print("Total weather rows (expanded to stations):", len(wx))


Unique grid cells to fetch: 26
⚠️  Cell 1360_-2464 failed
⚠️  Cell 1360_-2465 failed
⚠️  Cell 1360_-2466 failed
⚠️  Cell 1361_-2463 failed
⚠️  Cell 1362_-2463 failed
⚠️  Cell 1362_-2464 failed
⚠️  Cell 1517_-2452 failed
Weather rows (cells): 166440 | cells covered: 19
Total weather rows (expanded to stations): 6333676


In [24]:
# === Merge trip data with hourly weather data ===

# If weather was fetched per station (or per grid cell then expanded to stations), merge on BOTH keys; otherwise fall back to hour-only.
if "start_station_id" in wx.columns:
    # one weather row per (station, hour)
    wx_keyed = wx.drop_duplicates(["start_station_id", "start_hour"])
    dfm = df.merge(wx_keyed, on=["start_station_id", "start_hour"], how="left", validate="m:1")
else:
    # single-city weather (median lat/lon)
    wx_keyed = wx.drop_duplicates(["start_hour"])
    dfm = df.merge(wx_keyed, on="start_hour", how="left", validate="m:1")

# quick sanity check
miss = dfm["temperature_2m"].isna().mean()*100
print(f"Merged | shape={dfm.shape} | weather coverage={100-miss:.1f}%")


Merged | shape=(17548339, 24) | weather coverage=94.1%


In [25]:
# === Feature Engineering: Create new weather-based features ===

#Temperature (°C) and category labels
dfm["temp_celsius"] = dfm["temperature_2m"]
dfm["temp_category"] = pd.cut(
    dfm["temp_celsius"],
    bins=[-99, 0, 10, 20, 25, 30, 99],
    labels=["freezing", "cold", "cool", "comfortable", "warm", "hot"]
)

#Dryness indicator (1 = dry, 0 = rain/snow)
rain = dfm["rain"].fillna(0)
snow = dfm["snowfall"].fillna(0)
dfm["is_dry"] = ((rain + snow) == 0).astype("int8")

#Wind speed (convert from m/s → km/h)
dfm["wind_kmh"] = dfm["wind_speed_10m"] * 3.6

# Sky condition from % cloud cover
dfm["sky_condition"] = pd.cut(
    dfm["cloud_cover"],
    bins=[-1, 25, 50, 75, 100.1],
    labels=["clear", "partly_cloudy", "cloudy", "overcast"]
)

#Visibility in kilometers
dfm["visibility_km"] = dfm["visibility"] / 1000.0

#Cycling Score (0–100) — higher = better biking conditions
dfm["cycling_score"] = (
    np.exp(-((dfm["temp_celsius"] - 20) / 10) ** 2) * 40 +   # comfortable temperature
    dfm["is_dry"] * 30 +                                     # dry weather
    (dfm["wind_kmh"] < 20).astype("int8") * 20 +             # low wind
    (dfm["visibility_km"] > 5).astype("int8") * 10           # good visibility
).clip(0, 100)  # ensure score stays within range

#Convert categorical columns for memory efficiency
dfm["temp_category"] = dfm["temp_category"].astype("category")
dfm["sky_condition"] = dfm["sky_condition"].astype("category")

print(f"Weather features added | Total columns: {dfm.shape}")
dfm.head()


Weather features added | Total columns: (17548339, 31)


,tripduration,starttime,stoptime,start_station_id,start_station_latitude,start_station_longitude,end_station_id,end_station_latitude,end_station_longitude,bikeid,...,relative_humidity_2m,cloud_cover,visibility,temp_celsius,temp_category,is_dry,wind_kmh,sky_condition,visibility_km,cycling_score
0,970,2018-01-01 13:50:57.434,2018-01-01 14:07:08.186,72.0,40.767272,-73.993929,505.0,40.749013,-73.988484,31956,...,30.0,0.0,None,-8.1,freezing,1,73.439995,clear,NaN,30.014887
1,723,2018-01-01 15:33:30.182,2018-01-01 15:45:33.341,72.0,40.767272,-73.993929,3255.0,40.750585,-73.994685,32536,...,24.0,55.0,None,-6.9,freezing,1,70.919998,cloudy,NaN,30.028805
2,496,2018-01-01 15:39:18.337,2018-01-01 15:47:35.172,72.0,40.767272,-73.993929,525.0,40.755942,-74.002116,16069,...,24.0,55.0,None,-6.9,freezing,1,70.919998,cloudy,NaN,30.028805
3,306,2018-01-01 15:40:13.372,2018-01-01 15:45:20.191,72.0,40.767272,-73.993929,447.0,40.763707,-73.985162,31781,...,24.0,55.0,None,-6.9,freezing,1,70.919998,cloudy,NaN,30.028805
4,306,2018-01-01 18:14:51.568,2018-01-01 18:19:57.642,72.0,40.767272,-73.993929,3356.0,40.774667,-73.984706,30319,...,39.0,23.0,None,-10.0,freezing,1,26.639999,clear,NaN,30.004936
